# Final Project: Fuzzy matching of F1 data and deduplication

#### Part 1: Setup

##### Create the BQ datasets


In [14]:
!pip install --quiet google-cloud-bigquery
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project="saffatandsourik")


In [15]:
project_id = "saffatandsourik"
dataset = "formula1_fin"
region = "us-central1"
connection_id = "vertex-connection" # BQ requires a connection to call the model in Vertex
embedding_model = "text-embedding-005"  # latest gecko embeddings model as of 04/18/25
gemini_model = "gemini-2.5-flash-preview-04-17" # latest gemini flash model as of 04/18/25

In [16]:
from google.cloud import bigquery

bq_client = bigquery.Client()

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset saffatandsourik.formula1_fin


# Create Vertex AI Connection in BigQuery

establish a connection between BigQuery and Vertex AI to allow embedding model inference directly from SQL

In [17]:
!bq mk --connection --location=$region --project_id=$project_id \
    --connection_type=CLOUD_RESOURCE $connection_id

BigQuery error in mk operation: Already Exists: Connection projects/589546074977/locations/us-
central1/connections/vertex-connection


## Confirm Connection Exists

In [18]:
!bq mk --connection --location=us-central1 \
  --project_id=saffatandsourik \
  --connection_type=CLOUD_RESOURCE vertex-connection


BigQuery error in mk operation: Already Exists: Connection projects/589546074977/locations/us-
central1/connections/vertex-connection


In [19]:
!gcloud projects list --filter="projectId=saffatandsourik"


PROJECT_ID       NAME             PROJECT_NUMBER
saffatandsourik  saffatandsourik  589546074977


In [20]:
!bq show --connection 589546074977.us-central1.vertex-connection


Connection 589546074977.us-central1.vertex-connection

                     name                      friendlyName   description    Last modified         type        hasCredential                                            properties                                            
 -------------------------------------------- -------------- ------------- ----------------- ---------------- --------------- ----------------------------------------------------------------------------------------------- 
  589546074977.us-central1.vertex-connection                                25 Apr 19:49:24   CLOUD_RESOURCE   False           {"serviceAccountId": "bqcx-589546074977-bbxy@gcp-sa-bigquery-condel.iam.gserviceaccount.com"}  



## assign roles/aiplatform.admin to the BigQuery service account so it can access Vertex AI models during embedding generation

In [21]:
!gcloud projects add-iam-policy-binding saffatandsourik \
  --member="serviceAccount:bqcx-589546074977-bbxy@gcp-sa-bigquery-condel.iam.gserviceaccount.com" \
  --role="roles/aiplatform.admin"


Updated IAM policy for project [saffatandsourik].
bindings:
- members:
  - serviceAccount:bqcx-589546074977-bbxy@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/aiplatform.admin
- members:
  - serviceAccount:service-589546074977@gcp-sa-vertex-nb.iam.gserviceaccount.com
  role: roles/aiplatform.colabServiceAgent
- members:
  - serviceAccount:service-589546074977@gcp-sa-aiplatform-vm.iam.gserviceaccount.com
  role: roles/aiplatform.notebookServiceAgent
- members:
  - serviceAccount:service-589546074977@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-589546074977-bbxy@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  - user:rezasaffat@gmail.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:service-589546074977@gcp-sa-cloudaicompanion.iam.gserviceaccount.com
  role: roles/cloudaicompanion.serviceAgent
- members:
  - serviceAccount:service-589546074977@compute-system.iam.gserviceaccount.com
  role: 

## Create Remote Embedding Model
This model uses the latest text-embedding-005 endpoint to convert text fields into vector embeddings that can be used for fuzzy matching

In [22]:
%%bigquery

CREATE OR REPLACE MODEL `saffatandsourik.formula1_fin.embedding_model`
REMOTE WITH CONNECTION `saffatandsourik.us-central1.vertex-connection`
OPTIONS (endpoint = 'text-embedding-005');



Query is running:   0%|          |

""


## Generate Embeddings for Ergast Drivers
 Extract and concatenate driver name and nationality into a string and generate embeddings using the embedding_model. These embeddings will later be compared with OpenF1 driver embeddings to find close matches.

In [23]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.ergast_embeddings` AS (

  WITH ergast_content AS (
    SELECT
      permanent_number AS driver_id,
      CONCAT(LOWER(given_name), ' ', LOWER(family_name), ' ', LOWER(nationality)) AS content
    FROM `saffatandsourik.dbt_formula1_stg.ergast_drivers`
    WHERE permanent_number IS NOT NULL
  )

  SELECT
    driver_id,
    content,
    ml_generate_embedding_result AS embedding
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `saffatandsourik.formula1_fin.embedding_model`,
      (SELECT driver_id, content FROM ergast_content WHERE content IS NOT NULL),
      STRUCT('CLUSTERING' AS task_type)
    )

);


Query is running:   0%|          |

""


## Generate Embeddings for OpenF1 Drivers
Similarly, we create text strings for OpenF1 drivers and generate embeddings. This parallel structure allows us to compare drivers from two sources in vector space.

In [24]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.openf1_embeddings` AS (

  WITH openf1_content AS (
    SELECT
      driver_number,
      CONCAT(LOWER(first_name), ' ', LOWER(last_name), ' ', LOWER(country_code)) AS content
    FROM `saffatandsourik.dbt_formula1_stg.drivers_openf1`
  )

  SELECT
    driver_number,
    content,
    ml_generate_embedding_result AS embedding
  FROM
    ML.GENERATE_EMBEDDING(
      MODEL `saffatandsourik.formula1_fin.embedding_model`,
      (SELECT driver_number, content FROM openf1_content WHERE content IS NOT NULL),
      STRUCT('CLUSTERING' AS task_type)
    )

);


Query is running:   0%|          |

""


## Match Drivers Using Cosine Similarity
Use ML.DISTANCE with cosine similarity to identify drivers from OpenF1 and Ergast datasets who have similar embeddings. Only pairs with distance below 0.25 are considered valid matches.

### We tried 0.25, .1, and .3 and it gave poor results (inconsistent rows and not enough matches). so instead we will try .35

### .5 took too long to run

In [66]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.driver_matches` AS
SELECT
  openf1.driver_number,
  ergast.driver_id,
  ML.DISTANCE(openf1.embedding, ergast.embedding, 'COSINE') AS cosine_distance
FROM `saffatandsourik.formula1_fin.openf1_embeddings` AS openf1
JOIN `saffatandsourik.formula1_fin.ergast_embeddings` AS ergast
ON ML.DISTANCE(openf1.embedding, ergast.embedding, 'COSINE') < 0.35;


Query is running:   0%|          |

""


## Find Unmatched OpenF1 Drivers
We identify OpenF1 drivers that did not match with any Ergast driver. These will be assigned new driver_ids to preserve uniqueness.

In [67]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.unmatched_openf1` AS
SELECT *
FROM `saffatandsourik.dbt_formula1_stg.drivers_openf1`
WHERE driver_number NOT IN (
  SELECT driver_number FROM `saffatandsourik.formula1_fin.driver_matches`
);


Query is running:   0%|          |

""


## Assign New IDs to Unmatched Drivers

In [68]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.unmatched_openf1_numbered` AS
WITH max_id AS (
  SELECT MAX(driver_id) AS max_driver_id
  FROM `saffatandsourik.formula1_fin.ergast_embeddings`
),
deduplicated AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY LOWER(first_name), LOWER(last_name)
           ORDER BY _load_time DESC
         ) AS rn
  FROM `saffatandsourik.formula1_fin.unmatched_openf1`
),
numbered AS (
  SELECT *,
         ROW_NUMBER() OVER () + (SELECT max_driver_id FROM max_id) AS driver_id
  FROM deduplicated
  WHERE rn = 1
)

SELECT * FROM numbered;


Query is running:   0%|          |

""


## Combine Matched and Unmatched Drivers
We merge the Ergast drivers with the newly numbered unmatched OpenF1 drivers to form a unified driver table with consistent formatting

In [69]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.final_drivers` AS

SELECT
  permanent_number AS driver_id,
  CONCAT(UPPER(SUBSTRING(given_name, 1, 1)), LOWER(SUBSTRING(given_name, 2))) AS first_name,
  CONCAT(UPPER(SUBSTRING(family_name, 1, 1)), LOWER(SUBSTRING(family_name, 2))) AS last_name,
  code AS name_acronym,
  _data_source,
  _load_time
FROM `saffatandsourik.dbt_formula1_stg.ergast_drivers`
WHERE permanent_number IS NOT NULL

UNION ALL

SELECT
  driver_id,
  CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2))) AS first_name,
  CONCAT(UPPER(SUBSTRING(last_name, 1, 1)), LOWER(SUBSTRING(last_name, 2))) AS last_name,
  name_acronym,
  _data_source,
  _load_time
FROM `saffatandsourik.formula1_fin.unmatched_openf1_numbered`;


Query is running:   0%|          |

""


## Check for Duplicate Driver IDs
We verify that all driver_ids are unique

In [70]:
%%bigquery
SELECT driver_id, COUNT(*) as count
FROM `saffatandsourik.formula1_fin.final_drivers`
GROUP BY driver_id
HAVING count > 1;


Query is running:   0%|          |

Downloading:   0%|          |

,driver_id,count
0,2,14
1,3,14
2,4,14
3,10,14
4,11,14
5,14,14
6,16,14
7,18,14
8,20,14
9,21,7


## Check for repeating first names

In [71]:
%%bigquery
SELECT
  first_name,
  last_name,
  COUNT(*) AS count
FROM `saffatandsourik.formula1_fin.final_drivers`
GROUP BY first_name, last_name
HAVING COUNT(*) > 1
ORDER BY count DESC;


Query is running:   0%|          |

Downloading:   0%|          |

,first_name,last_name,count
0,Logan,Sargeant,14
1,Daniel,Ricciardo,14
2,Lando,Norris,14
3,Pierre,Gasly,14
4,Sergio,Pérez,14
5,Fernando,Alonso,14
6,Charles,Leclerc,14
7,Lance,Stroll,14
8,Kevin,Magnussen,14
9,Yuki,Tsunoda,14


## Deduplicate Final Driver Table
Since there were duplicates, we use ROW_NUMBER() to retain only the most recent entry (based on _load_time) for each unique driver_id and name combination

In [72]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik..formula1_fin.final_drivers` AS
WITH max_id AS (
  SELECT MAX(driver_id) AS max_driver_id
  FROM `saffatandsourik.formula1_fin.ergast_embeddings`
),
numbered AS (
  SELECT *,
         ROW_NUMBER() OVER (PARTITION BY first_name, last_name ORDER BY _load_time DESC) AS rn,
         ROW_NUMBER() OVER () + (SELECT max_driver_id FROM max_id) AS driver_id
  FROM `saffatandsourik.formula1_fin.unmatched_openf1`
)
SELECT *
FROM numbered
WHERE rn = 1;


Query is running:   0%|          |

""


In [73]:
%%bigquery
CREATE OR REPLACE TABLE `saffatandsourik.formula1_fin.final_drivers` AS
WITH unioned AS (
  SELECT
    permanent_number AS driver_id,
    CONCAT(UPPER(SUBSTRING(given_name, 1, 1)), LOWER(SUBSTRING(given_name, 2))) AS first_name,
    CONCAT(UPPER(SUBSTRING(family_name, 1, 1)), LOWER(SUBSTRING(family_name, 2))) AS last_name,
    code AS name_acronym,
    _data_source,
    _load_time
  FROM `saffatandsourik.dbt_formula1_stg.ergast_drivers`
  WHERE permanent_number IS NOT NULL

  UNION ALL

  SELECT
    driver_id,
    CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2))) AS first_name,
    CONCAT(UPPER(SUBSTRING(last_name, 1, 1)), LOWER(SUBSTRING(last_name, 2))) AS last_name,
    name_acronym,
    _data_source,
    _load_time
  FROM `saffatandsourik.formula1_fin.unmatched_openf1_numbered`
),

deduplicated AS (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY driver_id, first_name, last_name, name_acronym
           ORDER BY _load_time DESC
         ) AS row_num
  FROM unioned
)

SELECT *
FROM deduplicated
WHERE row_num = 1;


Query is running:   0%|          |

""


In [74]:
%%bigquery
SELECT driver_id, COUNT(*) as count
FROM `saffatandsourik.formula1_fin.final_drivers`
GROUP BY driver_id
HAVING count > 1;


Query is running:   0%|          |

Downloading: |          |

,driver_id,count


## Comparing with the old drivers table

In [75]:
%%bigquery
SELECT
  (SELECT COUNT(*) FROM `saffatandsourik.dbt_formula1_int.Drivers`) AS old_table_count,
  (SELECT COUNT(*) FROM `saffatandsourik.formula1_fin.final_drivers`) AS new_table_count;


Query is running:   0%|          |

Downloading:   0%|          |

,old_table_count,new_table_count
0,75,47


## Which drivers are newly found?

In [76]:
%%bigquery
WITH old_names AS (
  SELECT DISTINCT
    CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2))) AS first_name,
    CONCAT(UPPER(SUBSTRING(last_name, 1, 1)), LOWER(SUBSTRING(last_name, 2))) AS last_name
  FROM `saffatandsourik.dbt_formula1_int.Drivers`
),

new_names AS (
  SELECT DISTINCT
    CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2))) AS first_name,
    CONCAT(UPPER(SUBSTRING(last_name, 1, 1)), LOWER(SUBSTRING(last_name, 2))) AS last_name
  FROM `saffatandsourik.formula1_fin.final_drivers`
)

SELECT n.first_name, n.last_name
FROM new_names n
LEFT JOIN old_names o
  ON o.first_name = n.first_name AND o.last_name = n.last_name
WHERE o.first_name IS NULL


Query is running:   0%|          |

Downloading:   0%|          |

,first_name,last_name
0,Ryo,Hirakawa
1,Nico,Hülkenberg


## Which drivers are missing?


Some drivers appear to be missing, but this is mostly because they are spelled slightly different compared to the original table.

In [77]:
%%bigquery
WITH old_names AS (
  SELECT DISTINCT
    CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2))) AS first_name,
    CONCAT(UPPER(SUBSTRING(last_name, 1, 1)), LOWER(SUBSTRING(last_name, 2))) AS last_name
  FROM `saffatandsourik.dbt_formula1_int.Drivers`
),

new_names AS (
  SELECT DISTINCT
    CONCAT(UPPER(SUBSTRING(first_name, 1, 1)), LOWER(SUBSTRING(first_name, 2))) AS first_name,
    CONCAT(UPPER(SUBSTRING(last_name, 1, 1)), LOWER(SUBSTRING(last_name, 2))) AS last_name
  FROM `saffatandsourik.formula1_fin.final_drivers`
)

SELECT o.first_name, o.last_name
FROM old_names o
LEFT JOIN new_names n
  ON o.first_name = n.first_name AND o.last_name = n.last_name
WHERE n.first_name IS NULL


Query is running:   0%|          |

Downloading:   0%|          |

,first_name,last_name
0,Nico,Hulkenberg
1,Zak,O'sullivan
2,Nyck,Vries
3,Sophia,Floersch
4,Zhou,Guanyu
5,Roberto,Faria
6,Sergio,Perez
7,Patricio,O'ward
8,Maxwell,Esterson
9,Ayumu,Iwasa


Conclusion

In this project, we implemented fuzzy matching of F1 drivers using text embeddings. Instead of relying on exact keys or in-context language model prompting, we used BigQuery’s built-in `ml.generate_embedding()` to convert driver name + nationality into high-dimensional vectors. We then compared these vectors using `ML.DISTANCE` with cosine similarity to find semantically similar records between two datasets (`ergast` and `openf1`).

Compared to SQL joins or prompt-based matching, this embedding-based approach:
- Avoids hallucinations and inconsistencies
- Scales better for large datasets
- Provides a quantifiable threshold for similarity


However, compared to our manual approach, the fuzzy matching created less matches . This isn't the biggest issue because if our data were to scale dramatically, then it would be much easier to use fuzzy matching.

This method proved especially useful for matching inconsistent text (e.g., names with spelling differences or different formats). I would recommend incorporating embedding-based matching into the `int` layer of the pipeline when standardizing entities across sources, particularly when primary keys are missing or inconsistent.